## Purpose
This is to read raw data and merge/organize for dataloader.
The output format of sales is expected to be a dataframe with rows for timesteps, and columns as the time series id that needs to be predicted. 
Thus, the sales dataframe should be 1941x30490 for item-by-item organization. 
If add aggregations, the shape should be 1941x42840.

To distinguish the timeseries, we need save another dataframe as a header to map the timeseries id to its `state_id, store_id, cat_id, dept_id, item_id`.


This type of organization is good for channel-independent algorithms, which considers each time-series as an independent channel.

In [1]:
import logging
import pathlib
import joblib
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder


In [2]:
!pip install git+https://github.com/YangyangFu/transformer-time-series@main

/bin/bash: /home/yyf/miniconda3/envs/tts-tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  Cloning https://github.com/YangyangFu/transformer-time-series (to revision main) to /tmp/pip-req-build-5es18hjq
  Running command git clone --filter=blob:none --quiet https://github.com/YangyangFu/transformer-time-series /tmp/pip-req-build-5es18hjq
  Resolved https://github.com/YangyangFu/transformer-time-series to commit e7c89c69791834a63b2a0adb76813f3372143e6e
  Preparing metadata (setup.py) ... done


In [3]:
from tsl.utils.utils import reduce_mem_usage

2023-09-13 18:48:04.780156: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-13 18:48:04.803906: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-13 18:48:05.155560: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
def dump(df, name, path):
    df = reduce_mem_usage(df)
    save_dir = pathlib.Path(path)
    if not save_dir.exists():
        save_dir.mkdir(parents=True)
    joblib.dump(df, save_dir / f'{name}.joblib', compress=True)

dump_dir = os.path.join('./m5', 'individual')

In [5]:
data_dir = "./m5"

sales_file = "sales_train_evaluation.csv"
calendar_file = "calendar.csv"
price_file = "sell_prices.csv"


In [6]:
sales = pd.read_csv(os.path.join(data_dir, sales_file))
calendar = pd.read_csv(os.path.join(data_dir, calendar_file), parse_dates=["date"])
price = pd.read_csv(os.path.join(data_dir, price_file))

In [7]:
print(sales.columns)
print(calendar.columns)
print(price.columns)
print(sales.shape, calendar.shape, price.shape)

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1',
       'd_2', 'd_3', 'd_4',
       ...
       'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938',
       'd_1939', 'd_1940', 'd_1941'],
      dtype='object', length=1947)
Index(['date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'd',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI'],
      dtype='object')
Index(['store_id', 'item_id', 'wm_yr_wk', 'sell_price'], dtype='object')
(30490, 1947) (1969, 14) (6841121, 4)


## Global time features and time series

In [8]:
id_cols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']


In [9]:
s = sales.drop(id_cols, axis=1)
s.index = sales['id']
s = s.transpose()
s = s.reset_index()
s = s.rename(columns={'index': 'd'})
s.head()

id,d,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_009_CA_1_evaluation,...,FOODS_3_818_WI_3_evaluation,FOODS_3_819_WI_3_evaluation,FOODS_3_820_WI_3_evaluation,FOODS_3_821_WI_3_evaluation,FOODS_3_822_WI_3_evaluation,FOODS_3_823_WI_3_evaluation,FOODS_3_824_WI_3_evaluation,FOODS_3_825_WI_3_evaluation,FOODS_3_826_WI_3_evaluation,FOODS_3_827_WI_3_evaluation
0,d_1,0,0,0,0,0,0,0,12,2,...,0,14,1,0,4,0,0,0,0,0
1,d_2,0,0,0,0,0,0,0,15,0,...,0,11,1,0,4,0,0,6,0,0
2,d_3,0,0,0,0,0,0,0,0,7,...,0,5,1,0,2,2,0,0,0,0
3,d_4,0,0,0,0,0,0,0,0,3,...,0,6,1,0,5,2,0,2,0,0
4,d_5,0,0,0,0,0,0,0,0,0,...,0,5,1,0,2,0,0,2,0,0


In [10]:
s = s.merge(calendar[['d', 'date']], how = 'outer')
s = s.drop(['d'], axis=1)
s.index = s['date']
s = s.drop(['date'], axis=1)

In [11]:
s.shape, s.columns

((1969, 30490),
 Index(['HOBBIES_1_001_CA_1_evaluation', 'HOBBIES_1_002_CA_1_evaluation',
        'HOBBIES_1_003_CA_1_evaluation', 'HOBBIES_1_004_CA_1_evaluation',
        'HOBBIES_1_005_CA_1_evaluation', 'HOBBIES_1_006_CA_1_evaluation',
        'HOBBIES_1_007_CA_1_evaluation', 'HOBBIES_1_008_CA_1_evaluation',
        'HOBBIES_1_009_CA_1_evaluation', 'HOBBIES_1_010_CA_1_evaluation',
        ...
        'FOODS_3_818_WI_3_evaluation', 'FOODS_3_819_WI_3_evaluation',
        'FOODS_3_820_WI_3_evaluation', 'FOODS_3_821_WI_3_evaluation',
        'FOODS_3_822_WI_3_evaluation', 'FOODS_3_823_WI_3_evaluation',
        'FOODS_3_824_WI_3_evaluation', 'FOODS_3_825_WI_3_evaluation',
        'FOODS_3_826_WI_3_evaluation', 'FOODS_3_827_WI_3_evaluation'],
       dtype='object', length=30490))

In [12]:
s.head()

,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_010_CA_1_evaluation,...,FOODS_3_818_WI_3_evaluation,FOODS_3_819_WI_3_evaluation,FOODS_3_820_WI_3_evaluation,FOODS_3_821_WI_3_evaluation,FOODS_3_822_WI_3_evaluation,FOODS_3_823_WI_3_evaluation,FOODS_3_824_WI_3_evaluation,FOODS_3_825_WI_3_evaluation,FOODS_3_826_WI_3_evaluation,FOODS_3_827_WI_3_evaluation
date,,,,,,,,,,,,,,,,,,,,,
2011-01-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,2.0,0.0,...,0.0,14.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
2011-01-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,...,0.0,11.0,1.0,0.0,4.0,0.0,0.0,6.0,0.0,0.0
2011-01-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,...,0.0,5.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0
2011-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,6.0,1.0,0.0,5.0,2.0,0.0,2.0,0.0,0.0
2011-02-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0


In [13]:
dump(s, 'ts', dump_dir)

Mem. usage decreased to 114.52 Mb (75.0% reduction)


In [14]:
del s

## Local features

### Time-invariant local features
Time-invariant local features refer to the features that are dependent on the time series itself but not on time, such as `item_id, store_id`, etc.

In [15]:
items = sales[id_cols]
items.head()

,id,item_id,dept_id,cat_id,store_id,state_id
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA


Need convert to integers using label encoding

In [16]:
ids = items.copy()
label_encoders = {}
for col in ids.columns:
    encoder = LabelEncoder()
    ids[col] = encoder.fit_transform(ids[col])
    label_encoders[col] = encoder

ids.head()

,id,item_id,dept_id,cat_id,store_id,state_id
0,14370,1437,3,1,0,0
1,14380,1438,3,1,0,0
2,14390,1439,3,1,0,0
3,14400,1440,3,1,0,0
4,14410,1441,3,1,0,0


In [17]:
items.head()

,id,item_id,dept_id,cat_id,store_id,state_id
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA


In [18]:
dump(ids, 'local_invariant', dump_dir)

Mem. usage decreased to  0.23 Mb (83.3% reduction)


### Time-variant local features

local features that is time variant are `SNAP`, `item sell price`, etc

snap is a local feature based on states. it will be different for each time series.

the targets are each item in each store in each state, thus, each item should have their own SNAP feature based on the states it locates.

here I create a snap table for each time series, so that during training, we can grab the snap for corresponding target.

In [19]:
snap = calendar[['date', 'snap_CA', 'snap_TX', 'snap_WI']]
snap = snap.rename(columns={'snap_CA':'CA',
             'snap_TX':'TX',
             'snap_WI':'WI'})
snap = pd.concat([snap, pd.DataFrame(columns=items['id'])])
for idx, state in zip(items['id'], items['state_id']):
    snap[idx] = snap[state]


In [20]:
snap.index = snap['date']

In [21]:
snap = snap.drop(['CA', 'TX', 'WI', 'date'], axis=1)

In [22]:
snap.shape, snap.columns

((1969, 30490),
 Index(['HOBBIES_1_001_CA_1_evaluation', 'HOBBIES_1_002_CA_1_evaluation',
        'HOBBIES_1_003_CA_1_evaluation', 'HOBBIES_1_004_CA_1_evaluation',
        'HOBBIES_1_005_CA_1_evaluation', 'HOBBIES_1_006_CA_1_evaluation',
        'HOBBIES_1_007_CA_1_evaluation', 'HOBBIES_1_008_CA_1_evaluation',
        'HOBBIES_1_009_CA_1_evaluation', 'HOBBIES_1_010_CA_1_evaluation',
        ...
        'FOODS_3_818_WI_3_evaluation', 'FOODS_3_819_WI_3_evaluation',
        'FOODS_3_820_WI_3_evaluation', 'FOODS_3_821_WI_3_evaluation',
        'FOODS_3_822_WI_3_evaluation', 'FOODS_3_823_WI_3_evaluation',
        'FOODS_3_824_WI_3_evaluation', 'FOODS_3_825_WI_3_evaluation',
        'FOODS_3_826_WI_3_evaluation', 'FOODS_3_827_WI_3_evaluation'],
       dtype='object', length=30490))

In [23]:
snap.head()

,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_010_CA_1_evaluation,...,FOODS_3_818_WI_3_evaluation,FOODS_3_819_WI_3_evaluation,FOODS_3_820_WI_3_evaluation,FOODS_3_821_WI_3_evaluation,FOODS_3_822_WI_3_evaluation,FOODS_3_823_WI_3_evaluation,FOODS_3_824_WI_3_evaluation,FOODS_3_825_WI_3_evaluation,FOODS_3_826_WI_3_evaluation,FOODS_3_827_WI_3_evaluation
date,,,,,,,,,,,,,,,,,,,,,
2011-01-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-02-01,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-02-02,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [24]:
dump(snap, 'local_variant_snap', dump_dir)

Mem. usage decreased to 114.52 Mb (75.0% reduction)


In [25]:
del snap

Price for each item, should serve as a numerical time-variant local feature.

The price is represented on a weekly basis, while the sales is represent on a daily basis. 
Thus, we need manipulate the weekly price to daily price.

In [26]:
price.head(), price.shape

(  store_id        item_id  wm_yr_wk  sell_price
 0     CA_1  HOBBIES_1_001     11325        9.58
 1     CA_1  HOBBIES_1_001     11326        9.58
 2     CA_1  HOBBIES_1_001     11327        8.26
 3     CA_1  HOBBIES_1_001     11328        8.26
 4     CA_1  HOBBIES_1_001     11329        8.26,
 (6841121, 4))

In [27]:
# some items are only for sale after a specific date.
# here we have a release date to consider this effect
releases = price.groupby(['store_id','item_id'])['wm_yr_wk'].min().reset_index()
releases.columns = ['store_id','item_id','wm_yr_wk']
weekday = calendar.groupby('wm_yr_wk')['date'].min().reset_index()
releases = releases.merge(weekday)
releases.columns = ['store_id','item_id','release_week', 'release_date']
releases.drop('release_week', axis=1, inplace=True)
releases.head()

,store_id,item_id,release_date
0,CA_1,FOODS_1_001,2011-01-29
1,CA_1,FOODS_1_002,2011-01-29
2,CA_1,FOODS_1_003,2011-01-29
3,CA_1,FOODS_1_005,2011-01-29
4,CA_1,FOODS_1_006,2011-01-29


In [28]:
price.columns
pr = price.merge(releases)
pr.columns, pr.head()

(Index(['store_id', 'item_id', 'wm_yr_wk', 'sell_price', 'release_date'], dtype='object'),
   store_id        item_id  wm_yr_wk  sell_price release_date
 0     CA_1  HOBBIES_1_001     11325        9.58   2013-07-13
 1     CA_1  HOBBIES_1_001     11326        9.58   2013-07-13
 2     CA_1  HOBBIES_1_001     11327        8.26   2013-07-13
 3     CA_1  HOBBIES_1_001     11328        8.26   2013-07-13
 4     CA_1  HOBBIES_1_001     11329        8.26   2013-07-13)

In [29]:
pr = pr.merge(calendar[['wm_yr_wk', 'd', 'date']])
pr = pr.merge(items)
pr.columns, pr.head()

(Index(['store_id', 'item_id', 'wm_yr_wk', 'sell_price', 'release_date', 'd',
        'date', 'id', 'dept_id', 'cat_id', 'state_id'],
       dtype='object'),
   store_id        item_id  wm_yr_wk  sell_price release_date      d  \
 0     CA_1  HOBBIES_1_001     11325        9.58   2013-07-13  d_897   
 1     CA_1  HOBBIES_1_001     11325        9.58   2013-07-13  d_898   
 2     CA_1  HOBBIES_1_001     11325        9.58   2013-07-13  d_899   
 3     CA_1  HOBBIES_1_001     11325        9.58   2013-07-13  d_900   
 4     CA_1  HOBBIES_1_001     11325        9.58   2013-07-13  d_901   
 
         date                             id    dept_id   cat_id state_id  
 0 2013-07-13  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1  HOBBIES       CA  
 1 2013-07-14  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1  HOBBIES       CA  
 2 2013-07-15  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1  HOBBIES       CA  
 3 2013-07-16  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1  HOBBIES       CA  
 4 2013-07-17  HOBBIES_1_001

In [30]:
pr.head(), items.head()

(  store_id        item_id  wm_yr_wk  sell_price release_date      d  \
 0     CA_1  HOBBIES_1_001     11325        9.58   2013-07-13  d_897   
 1     CA_1  HOBBIES_1_001     11325        9.58   2013-07-13  d_898   
 2     CA_1  HOBBIES_1_001     11325        9.58   2013-07-13  d_899   
 3     CA_1  HOBBIES_1_001     11325        9.58   2013-07-13  d_900   
 4     CA_1  HOBBIES_1_001     11325        9.58   2013-07-13  d_901   
 
         date                             id    dept_id   cat_id state_id  
 0 2013-07-13  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1  HOBBIES       CA  
 1 2013-07-14  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1  HOBBIES       CA  
 2 2013-07-15  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1  HOBBIES       CA  
 3 2013-07-16  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1  HOBBIES       CA  
 4 2013-07-17  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1  HOBBIES       CA  ,
                               id        item_id    dept_id   cat_id store_id  \
 0  HOBBIES_1_001_CA_1_evalu

In [31]:
#pr['released'] = (pr['date'] >= pr['release_date']).astype(int)
pr = pr.drop(['wm_yr_wk', 'item_id', 'store_id', 'sell_price', 'dept_id', 'cat_id','state_id'], axis=1)
pr = pr.pivot(index='date', columns='id', values='release_date')

In [32]:
pr.head()

id,FOODS_1_001_CA_1_evaluation,FOODS_1_001_CA_2_evaluation,FOODS_1_001_CA_3_evaluation,FOODS_1_001_CA_4_evaluation,FOODS_1_001_TX_1_evaluation,FOODS_1_001_TX_2_evaluation,FOODS_1_001_TX_3_evaluation,FOODS_1_001_WI_1_evaluation,FOODS_1_001_WI_2_evaluation,FOODS_1_001_WI_3_evaluation,...,HOUSEHOLD_2_516_CA_1_evaluation,HOUSEHOLD_2_516_CA_2_evaluation,HOUSEHOLD_2_516_CA_3_evaluation,HOUSEHOLD_2_516_CA_4_evaluation,HOUSEHOLD_2_516_TX_1_evaluation,HOUSEHOLD_2_516_TX_2_evaluation,HOUSEHOLD_2_516_TX_3_evaluation,HOUSEHOLD_2_516_WI_1_evaluation,HOUSEHOLD_2_516_WI_2_evaluation,HOUSEHOLD_2_516_WI_3_evaluation
date,,,,,,,,,,,,,,,,,,,,,
2011-01-29,2011-01-29,2011-01-29,2011-01-29,2011-01-29,2011-01-29,2011-01-29,NaT,2011-01-29,2011-01-29,2011-01-29,...,2011-01-29,NaT,2011-01-29,NaT,NaT,2011-01-29,2011-01-29,2011-01-29,NaT,2011-01-29
2011-01-30,2011-01-29,2011-01-29,2011-01-29,2011-01-29,2011-01-29,2011-01-29,NaT,2011-01-29,2011-01-29,2011-01-29,...,2011-01-29,NaT,2011-01-29,NaT,NaT,2011-01-29,2011-01-29,2011-01-29,NaT,2011-01-29
2011-01-31,2011-01-29,2011-01-29,2011-01-29,2011-01-29,2011-01-29,2011-01-29,NaT,2011-01-29,2011-01-29,2011-01-29,...,2011-01-29,NaT,2011-01-29,NaT,NaT,2011-01-29,2011-01-29,2011-01-29,NaT,2011-01-29
2011-02-01,2011-01-29,2011-01-29,2011-01-29,2011-01-29,2011-01-29,2011-01-29,NaT,2011-01-29,2011-01-29,2011-01-29,...,2011-01-29,NaT,2011-01-29,NaT,NaT,2011-01-29,2011-01-29,2011-01-29,NaT,2011-01-29
2011-02-02,2011-01-29,2011-01-29,2011-01-29,2011-01-29,2011-01-29,2011-01-29,NaT,2011-01-29,2011-01-29,2011-01-29,...,2011-01-29,NaT,2011-01-29,NaT,NaT,2011-01-29,2011-01-29,2011-01-29,NaT,2011-01-29


there are some `NaT` in the columns, checkout why???

In [33]:
pr.shape, pr.columns

((1969, 30490),
 Index(['FOODS_1_001_CA_1_evaluation', 'FOODS_1_001_CA_2_evaluation',
        'FOODS_1_001_CA_3_evaluation', 'FOODS_1_001_CA_4_evaluation',
        'FOODS_1_001_TX_1_evaluation', 'FOODS_1_001_TX_2_evaluation',
        'FOODS_1_001_TX_3_evaluation', 'FOODS_1_001_WI_1_evaluation',
        'FOODS_1_001_WI_2_evaluation', 'FOODS_1_001_WI_3_evaluation',
        ...
        'HOUSEHOLD_2_516_CA_1_evaluation', 'HOUSEHOLD_2_516_CA_2_evaluation',
        'HOUSEHOLD_2_516_CA_3_evaluation', 'HOUSEHOLD_2_516_CA_4_evaluation',
        'HOUSEHOLD_2_516_TX_1_evaluation', 'HOUSEHOLD_2_516_TX_2_evaluation',
        'HOUSEHOLD_2_516_TX_3_evaluation', 'HOUSEHOLD_2_516_WI_1_evaluation',
        'HOUSEHOLD_2_516_WI_2_evaluation', 'HOUSEHOLD_2_516_WI_3_evaluation'],
       dtype='object', name='id', length=30490))

In [34]:
pr = pr[items['id']]

In [35]:
pr = pr.apply(lambda x: x <= pr.index, axis=0)
pr = pr.astype(int)

In [36]:
pr.head(), pr.shape

(id          HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_002_CA_1_evaluation  \
 date                                                                       
 2011-01-29                              0                              0   
 2011-01-30                              0                              0   
 2011-01-31                              0                              0   
 2011-02-01                              0                              0   
 2011-02-02                              0                              0   
 
 id          HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_004_CA_1_evaluation  \
 date                                                                       
 2011-01-29                              0                              0   
 2011-01-30                              0                              0   
 2011-01-31                              0                              0   
 2011-02-01                              0                              0 

In [37]:
dump(pr, 'local_variant_release', dump_dir)
del pr

Mem. usage decreased to 57.27 Mb (87.5% reduction)


In [38]:
price.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [39]:
price = price.merge(calendar[['wm_yr_wk', 'd', 'date']])

In [40]:
price.head(), price.shape

(  store_id        item_id  wm_yr_wk  sell_price      d       date
 0     CA_1  HOBBIES_1_001     11325        9.58  d_897 2013-07-13
 1     CA_1  HOBBIES_1_001     11325        9.58  d_898 2013-07-14
 2     CA_1  HOBBIES_1_001     11325        9.58  d_899 2013-07-15
 3     CA_1  HOBBIES_1_001     11325        9.58  d_900 2013-07-16
 4     CA_1  HOBBIES_1_001     11325        9.58  d_901 2013-07-17,
 (47735397, 6))

In [41]:
price = price.merge(items)

In [42]:
price.head(), price.shape

(  store_id        item_id  wm_yr_wk  sell_price      d       date  \
 0     CA_1  HOBBIES_1_001     11325        9.58  d_897 2013-07-13   
 1     CA_1  HOBBIES_1_001     11325        9.58  d_898 2013-07-14   
 2     CA_1  HOBBIES_1_001     11325        9.58  d_899 2013-07-15   
 3     CA_1  HOBBIES_1_001     11325        9.58  d_900 2013-07-16   
 4     CA_1  HOBBIES_1_001     11325        9.58  d_901 2013-07-17   
 
                               id    dept_id   cat_id state_id  
 0  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1  HOBBIES       CA  
 1  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1  HOBBIES       CA  
 2  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1  HOBBIES       CA  
 3  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1  HOBBIES       CA  
 4  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1  HOBBIES       CA  ,
 (47735397, 10))

In [43]:
price = price.pivot(index='date', columns='id', values='sell_price')

In [44]:
price.head(), price.shape

(id          FOODS_1_001_CA_1_evaluation  FOODS_1_001_CA_2_evaluation  \
 date                                                                   
 2011-01-29                          2.0                          2.0   
 2011-01-30                          2.0                          2.0   
 2011-01-31                          2.0                          2.0   
 2011-02-01                          2.0                          2.0   
 2011-02-02                          2.0                          2.0   
 
 id          FOODS_1_001_CA_3_evaluation  FOODS_1_001_CA_4_evaluation  \
 date                                                                   
 2011-01-29                          2.0                          2.0   
 2011-01-30                          2.0                          2.0   
 2011-01-31                          2.0                          2.0   
 2011-02-01                          2.0                          2.0   
 2011-02-02                          2.0         

In [45]:
price = price[items['id']]
price.head()

id,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_010_CA_1_evaluation,...,FOODS_3_818_WI_3_evaluation,FOODS_3_819_WI_3_evaluation,FOODS_3_820_WI_3_evaluation,FOODS_3_821_WI_3_evaluation,FOODS_3_822_WI_3_evaluation,FOODS_3_823_WI_3_evaluation,FOODS_3_824_WI_3_evaluation,FOODS_3_825_WI_3_evaluation,FOODS_3_826_WI_3_evaluation,FOODS_3_827_WI_3_evaluation
date,,,,,,,,,,,,,,,,,,,,,
2011-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.46,1.56,3.17,...,NaN,1.98,1.57,NaN,3.98,2.5,2.47,4.0,NaN,NaN
2011-01-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.46,1.56,3.17,...,NaN,1.98,1.57,NaN,3.98,2.5,2.47,4.0,NaN,NaN
2011-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.46,1.56,3.17,...,NaN,1.98,1.57,NaN,3.98,2.5,2.47,4.0,NaN,NaN
2011-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.46,1.56,3.17,...,NaN,1.98,1.57,NaN,3.98,2.5,2.47,4.0,NaN,NaN
2011-02-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.46,1.56,3.17,...,NaN,1.98,1.57,NaN,3.98,2.5,2.47,4.0,NaN,NaN


In [46]:
price = price.fillna(value=0)
dump(price, 'local_variant_price', dump_dir)

Mem. usage decreased to 114.52 Mb (75.0% reduction)


In [47]:
del price


Note some items are not for sales within the given time period, thus the prices are `NAN`. 

As noticed, there are a lot of zeros/NAN in the price, which basically due to "out of stock" or "not released". However, we dont have information to indicate if a zero/NAN price is due to out of stock. 

Based on the original price data, we can see some items only have price info after specific date. We will set that date as the release date. 

The NANs for testing during `d1942-d1969` are due to the fact that the testing data is not openly accessible during competition.

## Move for Model

Finally, we move the processed to data to a final data preparation folder.

In [48]:
import shutil

dest_dir = '../datasets/m5/individual'
shutil.copytree(dump_dir, dest_dir, dirs_exist_ok=True)

# delete source folder
shutil.rmtree(dump_dir)
